# **1-Install required packages**

In [ ]:
%%capture
!pip install -q torch torchvision pillow faiss-cpu umap-learn matplotlib tqdm requests
!pip install -q git+https://github.com/openai/CLIP.git

In [ ]:
import os
import random
import numpy as np
import torch
import clip
from PIL import Image
import umap.umap_ as umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import faiss
from tqdm import tqdm

# **2-Create necessary directories**

In [ ]:
os.makedirs('/content/images', exist_ok=True)
os.makedirs('/content/output', exist_ok=True)

# **3-Download alternative flower dataset (from a reliable TensorFlow source)**

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
!wget -q --show-progress {dataset_url} -O /content/flower_dataset.tgz
!tar -xzf /content/flower_dataset.tgz -C /content/images/

# **4-Randomly select 200 images**

In [ ]:
all_files = []
for root, _, files in os.walk('/content/images/flower_photos'):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            all_files.append(os.path.join(root, file))

selected_files = random.sample(all_files, min(200, len(all_files)))

# Clean up
!rm /content/flower_dataset.tgz
print(f"\nNumber of images ready for processing: {len(selected_files)}")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# **5-Load CLIP model**

In [ ]:
model, preprocess = clip.load("ViT-B/32", device=device)
print("CLIP model loaded successfully.")

# **6-Extract embeddings**

In [ ]:
def extract_embeddings(image_paths, batch_size=32):
    embeddings = []
    valid_paths = []
    for i in tqdm(range(0, len(image_paths), batch_size), desc="Processing images", unit="batch"):
        batch_paths = image_paths[i:i+batch_size]
        batch_images = []
        for path in batch_paths:
            try:
                image = Image.open(path).convert("RGB")
                batch_images.append(preprocess(image))
                valid_paths.append(path)
            except Exception as e:
                print(f"Error processing {path}: {str(e)}")
        if batch_images:
            image_input = torch.stack(batch_images).to(device)
            with torch.no_grad():
                batch_embeddings = model.encode_image(image_input).cpu().numpy()
            embeddings.append(batch_embeddings)
    embeddings = np.vstack(embeddings)
    embeddings = normalize(embeddings, axis=1)
    return embeddings, valid_paths

embeddings, valid_paths = extract_embeddings(selected_files)
print(f"Extracted embeddings: {embeddings.shape}")

# **7-Dimensionality reduction with UMAP**

In [ ]:
reducer = umap.UMAP(
    n_components=2,
    metric='cosine',
    random_state=42,
    n_neighbors=15,
    min_dist=0.1,
    verbose=True
)

print("Reducing dimensions with UMAP...")
embeddings_2d = reducer.fit_transform(embeddings)
print("UMAP dimensionality reduction complete.")

# **8-Show advanced visualization with images on scatter**

In [ ]:
plt.figure(figsize=(20, 16), dpi=120)
ax = plt.gca()
ax.set_facecolor('#f5f5f5')
plt.grid(color='white', linestyle='--', linewidth=0.5)

for i, (x, y) in enumerate(embeddings_2d):
    try:
        img = Image.open(valid_paths[i])
        img.thumbnail((80, 80), Image.LANCZOS)
        ax.imshow(img, extent=(x-3, x+3, y-3, y+3), alpha=0.9)
        plt.text(x, y+4, os.path.basename(valid_paths[i]), fontsize=8, ha='center', alpha=0.7)
    except Exception as e:
        continue

plt.title('Semantic Map of Flower Images (UMAP + CLIP)', fontsize=16, pad=20)
plt.xlabel('UMAP Dimension 1', fontsize=12)
plt.ylabel('UMAP Dimension 2', fontsize=12)
plt.show()

def visualize_search_results(query_path, results):
    plt.figure(figsize=(18, 4))
    plt.subplot(1, 6, 1)
    query_img = Image.open(query_path)
    plt.imshow(query_img)
    plt.title('Query Image', fontsize=10)
    plt.axis('off')
    for i, result in enumerate(results[:5]):
        plt.subplot(1, 6, i+2)
        img = Image.open(result)
        plt.imshow(img)
        plt.title(f'Result {i+1}\n{os.path.basename(result)}', fontsize=8)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# **9-Build FAISS index for search**

In [ ]:
embeddings_faiss = embeddings.astype('float32')
dimension = embeddings_faiss.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(embeddings_faiss)
index.add(embeddings_faiss)

In [ ]:
query_path = random.choice(valid_paths)
query_image = Image.open(query_path)
query_input = preprocess(query_image).unsqueeze(0).to(device)
with torch.no_grad():
    query_embedding = model.encode_image(query_input).cpu().numpy()
query_embedding = normalize(query_embedding.reshape(1, -1)).astype('float32')
faiss.normalize_L2(query_embedding)
distances, indices = index.search(query_embedding, 5)
results = [valid_paths[i] for i in indices[0]]

In [ ]:
# Show search results
visualize_search_results(query_path, results)

# **10-Density visualization**

In [ ]:
plt.figure(figsize=(14, 10))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1],
           c=np.linalg.norm(embeddings, axis=1), cmap='viridis', alpha=0.7, s=100)
plt.colorbar(label='Embedding Norm')
for i in np.random.choice(len(valid_paths), min(20, len(valid_paths)), replace=False):
    plt.text(embeddings_2d[i, 0], embeddings_2d[i, 1],
            os.path.basename(valid_paths[i]), fontsize=8, alpha=0.8)
plt.title('Density of Image Embeddings', fontsize=16)
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.grid(True, alpha=0.2)
plt.show()